In [0]:
import os
import os.path
from os import path
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from matplotlib import patches
import keras

from PIL import Image
import pylab as pyl

from keras import models
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard

import time
import tensorflow as tf

Using TensorFlow backend.


In [0]:
base_dir = '/workdir/soler/SSMs_with_metergram/'
train_dir = '/workdir/soler/SSMs_with_metergram/Train/'
validation_dir = '/workdir/soler/SSMs_with_metergram/Validation/'
test_dir= '/workdir/soler/SSMs_with_metergram/Test/'


# Directory with our training SMJ_nonsolo pictures
train_SMJ_nonsolo_dir = '/workdir/soler/SSMs_with_metergram/Train/Non_segment'

# Directory with our training SMJ_solo pictures
train_SMJ_solo_dir = '/workdir/soler/SSMs_with_metergram/Train/Segment'


# Directory with our test SMJ_nonsolo pictures
validation_SMJ_nonsolo_dir = os.path.join(validation_dir, 'Non_segment')

# Directory with our test SMJ_solo pictures
validation_SMJ_solo_dir = os.path.join(validation_dir, 'Segment')

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        directory=train_dir,  # This is the source directory for training images
        target_size=(50,50),  # All images will be resized to 100*100
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 128 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        directory=validation_dir,
        target_size=(50, 50),
        batch_size=128,
        class_mode='binary')


test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(50, 50),
    batch_size=1,
    class_mode=None)



Found 138280 images belonging to 2 classes.
Found 13024 images belonging to 2 classes.
Found 77733 images belonging to 1 classes.


In [0]:
model = tf.keras.models.load_model('/workdir/soler/SSMdownbeatsyncwith-weights-improvement-17-0.80.h5')

model.evaluate_generator(generator=validation_generator,steps=1)

In [0]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()

In [0]:
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

77733/77733 [==============================] - 27711s 356ms/step


In [0]:
y_pred=[1 if y>=0.5 else 0 for y in pred]

In [0]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in y_pred]

print(predictions)

In [0]:
import pickle

In [0]:
with open('/workdir/soler/predictionsSSMwithdownbeatsync', 'wb') as f:
    pickle.dump(predictions, f)
